# Imports Block

In [43]:
# %pip install gender-guesser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 4.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [44]:
import numpy as np
import pandas as pd
import re
#import requests
import gender_guesser.detector as gender
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Data Cleaning 

## Import Data

In [2]:
data = pd.read_csv('../data/seattle_airbnb_listings.csv')

data.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,879181,In Jimi Hendrix's neighborhood 3,287172,Laura,Central Area,Atlantic,47.60096,-122.30190,Private room,53,30,99,2022-12-16,0.80,2,247,4,STR-OPLI-19-002301
1,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,159,2022-09-03,0.95,3,99,2,str-opli-19-002622
2,902609,Entire guest suite in West Seattle,3264544,Fabio,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,2,238,2023-03-21,1.95,5,206,27,STR-OPLI-19-002360
3,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,2,175,2023-01-15,1.14,9,356,20,STR-OPLI-19-003039
4,226536,Serene Room in Sunny Cottage near Discovery Park,209571,Cheryl,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,1,316,2022-12-12,2.28,1,209,53,STR-OPLI-19-000684
5,226677,Sunny Parisian room w/ best cooling memoryfoam...,30559,Angielena,Beacon Hill,Mid-Beacon Hill,47.55062,-122.31827,Private room,55,2,245,2023-03-11,1.77,9,363,51,STR-OPLI-19-003039
6,9531,The Adorable Sweet Orange Craftsman,31481,Cassie,West Seattle,Fairmount Park,47.55495,-122.38663,Entire home/apt,175,3,71,2022-12-28,0.52,2,228,11,STR-OPLI-19-002182
7,904057,Seattle Mini Home with Hot Tub and Private Deck,4678869,Errin,Ballard,West Woodland,47.66617,-122.36693,Entire home/apt,109,3,496,2023-03-20,4.04,1,274,40,STR-OPLI-19-000380
8,9534,The Coolest Tangerine Dream MIL!,31481,Cassie,West Seattle,Fairmount Park,47.55627,-122.38607,Entire home/apt,135,2,73,2023-03-09,0.54,2,231,13,STR-OPLI-19-002182
9,9596,"the down home , spacious, central and fab!",14942,Joyce,Other neighborhoods,Wallingford,47.65608,-122.33602,Entire home/apt,130,30,97,2020-09-28,0.68,3,65,0,STR -OPLI-19-002622


In [3]:
data.shape

(6376, 18)

In [4]:
data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,6.376000e+03,6.376000e+03,6376.000000,6376.000000,6376.000000,6376.000000,6376.000000,5289.000000,6376.000000,6376.000000,6376.000000
mean,2.538927e+17,1.321146e+08,47.626583,-122.334138,161.835790,12.364335,62.649467,2.137389,37.672208,181.394448,17.318381
std,3.398556e+17,1.487760e+08,0.049192,0.033081,160.151751,18.406114,101.318763,1.902515,99.954541,121.741392,23.230614
min,6.606000e+03,2.536000e+03,47.497920,-122.419050,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,2.571056e+07,1.671630e+07,47.602075,-122.356498,91.000000,2.000000,2.000000,0.610000,1.000000,74.000000,1.000000
50%,5.019875e+07,7.231629e+07,47.622580,-122.332818,130.000000,3.000000,21.000000,1.670000,2.000000,175.000000,7.000000
75%,6.452500e+17,1.985463e+08,47.664054,-122.311219,188.000000,30.000000,79.000000,3.160000,13.000000,295.000000,27.000000
max,8.530090e+17,5.064984e+08,47.734010,-122.240870,4000.000000,365.000000,1251.000000,22.700000,408.000000,365.000000,406.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6376 entries, 0 to 6375
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6376 non-null   int64  
 1   name                            6376 non-null   object 
 2   host_id                         6376 non-null   int64  
 3   host_name                       6376 non-null   object 
 4   neighbourhood_group             6376 non-null   object 
 5   neighbourhood                   6376 non-null   object 
 6   latitude                        6376 non-null   float64
 7   longitude                       6376 non-null   float64
 8   room_type                       6376 non-null   object 
 9   price                           6376 non-null   int64  
 10  minimum_nights                  6376 non-null   int64  
 11  number_of_reviews               6376 non-null   int64  
 12  last_review                     52

## Check the Dataframe for Duplicated Values

In [6]:
# check this dataframe for duplicates
data.duplicated().sum()

0

## Dealing with Missing Values

In [7]:
# next, lets check this dataframe for null values
data.isnull().sum()

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       1087
reviews_per_month                 1087
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
license                           1459
dtype: int64

In [8]:
# lets drop all properties that do not have value for 'license' column
# since, it is illigal to rent without license in Seattle, WA
data_without_nan_license = data.dropna(subset = ['license'])

data_without_nan_license.shape

(4917, 18)

In [9]:
data_without_nan_license.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                       373
reviews_per_month                 373
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
dtype: int64

In [10]:
# lets check number_of_reviews to see are their plcaes with overall zero reviews
zero_reviews = data_without_nan_license[data_without_nan_license['number_of_reviews'] == 0]

zero_reviews

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
366,6336564,Charming King Suite in Wallingford,13007578,Genevieve,Other neighborhoods,Wallingford,47.664470,-122.327280,Entire home/apt,120,2,0,NaN,NaN,1,119,0,STR-OPLI-22-001828
994,16544455,Peaceful Cottage in Friendly Seattle Neighborhood,19381549,Ivan,Lake City,Matthews Beach,47.714010,-122.285280,Entire home/apt,80,2,0,NaN,NaN,1,0,0,STR-OPLI-22-001466
1168,16996377,"View, Location, Large space and Comfort",26032511,Tracy,Queen Anne,East Queen Anne,47.630130,-122.345640,Entire home/apt,191,7,0,NaN,NaN,2,56,0,STR-OPLI-19-001330
1352,22772724,Phinney Ridge Craftsman,16065294,Cath And Nick,Other neighborhoods,Phinney Ridge,47.682280,-122.363120,Entire home/apt,464,7,0,NaN,NaN,1,231,0,STR-OPLI-22-001365
1374,22868081,Retreat in the City,6568776,Kyla,Beacon Hill,Mid-Beacon Hill,47.553803,-122.305254,Entire home/apt,200,2,0,NaN,NaN,1,42,0,STR-OPLI-23-000327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6331,739988181840452929,December in Seattle - x3Parking,14947641,Olivia,Other neighborhoods,Phinney Ridge,47.676950,-122.360410,Entire home/apt,300,2,0,NaN,NaN,1,362,0,STR-OPLI-21-000337
6341,740921669560120481,"Private Grande Homey Oasis, 2 homes, 1 Lot",117707804,Carrie,Other neighborhoods,Greenwood,47.703075,-122.361059,Entire home/apt,282,3,0,NaN,NaN,3,341,0,STR-OPLI-22-001410
6359,742668673884212971,Woodland Park's Cozy Place (Ground Unit),477279461,Jen,Other neighborhoods,Phinney Ridge,47.665060,-122.347850,Entire home/apt,189,2,0,NaN,NaN,17,151,0,STR-OPLI-22-000515
6360,742693157478096998,Woodland Park's Charming Retreat (Top Floor),477279461,Jen,Other neighborhoods,Wallingford,47.663370,-122.346120,Entire home/apt,307,2,0,NaN,NaN,17,139,0,STR-OPLI-22-000515


In [11]:
# so, their is 1087 properties that have zero reviews 
# and last_review has 1087 and reviews_per_month has 1087 null values
# so, it is likely that if property had zero reviews it got null value for these two columns
# lets make a funtion to check this
def check_null_values(df):
    zero_reviews = df[df['number_of_reviews'] == 0]
    non_matching_rows = zero_reviews[zero_reviews['last_review'].isnull() != zero_reviews['reviews_per_month'].isnull()]
    if non_matching_rows.empty:
        return "They all match."
    else:
        return non_matching_rows.index.tolist()

In [12]:
check_null_values(data_without_nan_license)

'They all match.'

In [13]:
# so, we can change all NaN values for 'reviews_per_month' to zero 
# since, there was no reviews for this property
def update_reviews_per_month(df):
    df.loc[df['number_of_reviews'] == 0, 'reviews_per_month'] = 0
    return df

In [14]:
data_without_nan_license_and_nan_rev_per_month = update_reviews_per_month(data_without_nan_license)

data_without_nan_license_and_nan_rev_per_month.shape

(4917, 18)

In [15]:
data_without_nan_license_and_nan_rev_per_month.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                       373
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
dtype: int64

## Data Formatting

### Handelling Text Data for 'licence' column

In [16]:
data_without_nan_license_and_nan_rev_per_month['license'].unique()

array(['STR-OPLI-19-002301', 'str-opli-19-002622', 'STR-OPLI-19-002360',
       ..., 'STR-OPLI-22-001581', 'STR-OPLI-22-001898',
       'STR-OPLI-22-001817'], dtype=object)

In [17]:
data_without_nan_license_and_nan_rev_per_month['license'].nunique()

3443

In [18]:
# check if only the same host has the same 'license' number
same_host_same_license = data_without_nan_license_and_nan_rev_per_month.groupby(['host_id', 'license']).size().reset_index(name='count')
result = (same_host_same_license.groupby('host_id')['count'].nunique() == 1).all()

result

False

**Note:** since, result is False, it means that there are some cases where different hosts have the same 'license' number. This could indicate a potential issue or inconsistency in the data. 

Also, **note:** locense have different fond, so lets use some text processing techniques.

In [19]:
def format_license(df):
    # Retrieve the 'license' column
    licenses = df['license']

    # Convert license values to lowercase
    licenses = licenses.str.lower()

    # Remove leading and trailing whitespaces
    licenses = licenses.str.strip()

    # Remove extra spaces within the license
    licenses = licenses.str.replace(r'\s+', ' ')

    # Remove non-alphanumeric characters
    licenses = licenses.apply(lambda x: re.sub(r'\W+', '', x))

    # Update the 'license' column in the DataFrame
    df['license'] = licenses

    return df

In [20]:
data_without_nan_nan_rpm_and_format_license = format_license(data_without_nan_license_and_nan_rev_per_month)

data_without_nan_nan_rpm_and_format_license.head()

/var/folders/xk/4css72c92cl2p_c60yft_4d00000gn/T/ipykernel_32223/2041462724.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  licenses = licenses.str.replace(r'\s+', ' ')
/var/folders/xk/4css72c92cl2p_c60yft_4d00000gn/T/ipykernel_32223/2041462724.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['license'] = licenses


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,879181,In Jimi Hendrix's neighborhood 3,287172,Laura,Central Area,Atlantic,47.60096,-122.30190,Private room,53,30,99,2022-12-16,0.80,2,247,4,stropli19002301
1,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,159,2022-09-03,0.95,3,99,2,stropli19002622
2,902609,Entire guest suite in West Seattle,3264544,Fabio,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,2,238,2023-03-21,1.95,5,206,27,stropli19002360
3,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,2,175,2023-01-15,1.14,9,356,20,stropli19003039
4,226536,Serene Room in Sunny Cottage near Discovery Park,209571,Cheryl,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,1,316,2022-12-12,2.28,1,209,53,stropli19000684


In [21]:
data_without_nan_nan_rpm_and_format_license['license'].nunique()

3423

In [22]:
# check if only the same host has the same 'license' number
same_host_same_license = data_without_nan_nan_rpm_and_format_license.groupby(['host_id', 'license']).size().reset_index(name='count')
result = (same_host_same_license.groupby('host_id')['count'].nunique() == 1).all()

result

False

In [23]:
duplicate_licenses = \
data_without_nan_nan_rpm_and_format_license[data_without_nan_nan_rpm_and_format_license['license'].\
                                            duplicated(keep = False)]

properties_with_duplicate_licenses = duplicate_licenses.groupby('license')['host_id'].unique()

properties_with_different_hosts = \
properties_with_duplicate_licenses[properties_with_duplicate_licenses.apply(lambda x: len(x) > 1)]

total_properties = len(properties_with_different_hosts)

print(f'Total properties with the same license but different host IDs: {total_properties}')

Total properties with the same license but different host IDs: 174


In [25]:
properties_with_different_hosts.head(10)

license
exempt             [103427, 31751507, 190880343, 41643403, 270104...
stropli19000047                                [10013531, 427025860]
stropli19000275                                [50608719, 190507384]
stropli19000433                               [212885663, 346462113]
stropli19000461                                 [4485958, 124853735]
stropli19000473                               [158715708, 403546810]
stropli19000476                               [487344448, 246393790]
stropli19000539                                [15942582, 142248716]
stropli19000635                                [36436545, 121688409]
stropli19000680                                  [26736911, 4754874]
Name: host_id, dtype: object

In [26]:
properties_with_different_hosts.tail(10)

license
stropli22001810                [484899828, 72316289]
stropli22001811                [484899828, 28222082]
stropli22001812    [484899828, 379735904, 310761349]
stropli22001813               [484899828, 379735904]
stropli22001814    [480513426, 484899828, 379735904]
stropli22001815    [122537749, 484899828, 379735904]
stropli22001932               [369696587, 369666149]
stropli22001983               [488392026, 437172354]
stropli23000098                [480679576, 29805789]
stropli23000266               [180545052, 107456887]
Name: host_id, dtype: object

In [27]:
# Filter for 'license' values that contain only numbers
numeric_license = data_without_nan_nan_rpm_and_format_license[data_without_nan_nan_rpm_and_format_license['license'].str.match('^\d+$')]

print(f"Numeric License Values: {numeric_license['license'].unique()}")

Numeric License Values: ['825676' '19000920' '817508' '838099' '19001138' '0008396460754365'
 '840989' '19000720' '0010136284' '0010133640' '19001149' '19000857'
 '19000514' '19000973' '19000313' '19000633' '12345' '7003' '604494726'
 '70036000276690010001' '6044606360010001' '839753' '19000047' '604484931'
 '0008395980754312' '837513' '19001052' '731255' '19001166' '839775'
 '837965']


In [28]:
# Filter for 'license' values that contain only letters
alphabetic_license = data_without_nan_nan_rpm_and_format_license[data_without_nan_nan_rpm_and_format_license['license'].str.match('^[a-zA-Z]+$')]

print(f"Alphabetic License Values: {alphabetic_license['license'].unique()}")

Alphabetic License Values: ['exempt' 'approvedbygovernment']


**Note:** license values can vary and may have different formats or naming conventions based on local regulations or the host's specific situation. Therefore, further investigation and reference to official sources are recommended to determine the exact meaning and significance of a specific license value in a given location.

So, for the sake of time for this project, I will assume that if property has 'license' then this license is valide. Hence, I will **drop 'license' column** from my analysis espatially since most of the license are unique.

In [29]:
data_without_nan_nan_rpm_and_format_license.shape

(4917, 18)

### Handelling Text Data for 'name' column

In [30]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    
    # Removing punctuation
    text = re.sub('[' + string.punctuation + ']', '', text)
    
    # Removing special characters
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    
    # Removing " w"
    text = text.replace(' w', '')
    
    # Tokenization
    tokens = text.split()
    
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Joining tokens back to text
    processed_text = ' '.join(tokens)
    
    return processed_text

In [31]:
# make a copy of the DataFrame
name_data_processed = data_without_nan_nan_rpm_and_format_license.copy()

In [32]:
# next, apply preprocessing to 'name' column in the copied DataFrame
name_data_processed['name_processed'] = name_data_processed['name'].apply(preprocess_text)

In [33]:
name_data_processed.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,name_processed
0,879181,In Jimi Hendrix's neighborhood 3,287172,Laura,Central Area,Atlantic,47.60096,-122.30190,Private room,53,30,99,2022-12-16,0.80,2,247,4,stropli19002301,jimi hendrix neighborhood 3
1,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,159,2022-09-03,0.95,3,99,2,stropli19002622,fab privat seattl urban cottag
2,902609,Entire guest suite in West Seattle,3264544,Fabio,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,2,238,2023-03-21,1.95,5,206,27,stropli19002360,entir guest suit inest seattl
3,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,2,175,2023-01-15,1.14,9,356,20,stropli19003039,gloriou sun room memori foamb
4,226536,Serene Room in Sunny Cottage near Discovery Park,209571,Cheryl,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,1,316,2022-12-12,2.28,1,209,53,stropli19000684,seren room sunni cottag near discoveri park
5,226677,Sunny Parisian room w/ best cooling memoryfoam...,30559,Angielena,Beacon Hill,Mid-Beacon Hill,47.55062,-122.31827,Private room,55,2,245,2023-03-11,1.77,9,363,51,stropli19003039,sunni parisian room best cool memoryfoam bed
6,9531,The Adorable Sweet Orange Craftsman,31481,Cassie,West Seattle,Fairmount Park,47.55495,-122.38663,Entire home/apt,175,3,71,2022-12-28,0.52,2,228,11,stropli19002182,ador sweet orang craftsman
7,904057,Seattle Mini Home with Hot Tub and Private Deck,4678869,Errin,Ballard,West Woodland,47.66617,-122.36693,Entire home/apt,109,3,496,2023-03-20,4.04,1,274,40,stropli19000380,seattl mini homeith hot tub privat deck
8,9534,The Coolest Tangerine Dream MIL!,31481,Cassie,West Seattle,Fairmount Park,47.55627,-122.38607,Entire home/apt,135,2,73,2023-03-09,0.54,2,231,13,stropli19002182,coolest tangerin dream mil
9,9596,"the down home , spacious, central and fab!",14942,Joyce,Other neighborhoods,Wallingford,47.65608,-122.33602,Entire home/apt,130,30,97,2020-09-28,0.68,3,65,0,stropli19002622,home spaciou central fab


### Changing 'host_name' column to Male or Female values (binary)

**Note:** this code worked before and now it stoped working

In [36]:
# def determine_gender(name):
#     response = requests.get(f"https://api.genderize.io?name={name}")
#     data = response.json()

#     if 'gender' in data:
#         return data['gender']
#     else:
#         return 'Unknown'

In [37]:
# # Example usage
# host_name = 'John'
# gender = determine_gender(host_name)
# print(f"The gender of {host_name} is: {gender}")

In [42]:
# # Apply the determine_gender function to each value in the 'host_name' column
# name_data_processed['host_gender'] = name_data_processed['host_name'].apply(determine_gender)

# # Print the updated DataFrame
# name_data_processed.head()

**Note:** The code relies on an external API to determine gender based on the name, which may have limitations or accuracy issues. Additionally, the execution time of the code may vary depending on the size of the DataFrame and the speed of the API response.

In [40]:
# do text data pre processing to make sure it is only one first name inclusded
name_data_processed['host_name'] = name_data_processed['host_name'].str.split().str[0]

name_data_processed['host_name']

0           Laura
1           Joyce
2           Fabio
3       Angielena
4          Cheryl
          ...    
6370        Julia
6372        Kiera
6373        Julia
6374       Karina
6375        Julia
Name: host_name, Length: 4917, dtype: object

In [52]:
# also check that name_data_processed['host_name'] has only letters
is_only_letters = name_data_processed['host_name'].str.isalpha().all()

print(f"The 'host_name' column contains only letters: {is_only_letters}")

The 'host_name' column contains only letters: False


In [55]:
# Create a boolean mask for rows with non-letter values
non_letter_mask = name_data_processed['host_name'].str.contains(r'[^a-zA-Z]', regex=True)

# Get the rows with non-letter values
rows_with_non_letters = name_data_processed[non_letter_mask]

# Print the rows with non-letter values
print(f"Rows with non-letter values: {rows_with_non_letters['host_name']}")

Rows with non-letter values: 191         Jean-Sébastien
1196        Jean-Sébastien
1274                   Mt.
1277                   Mt.
1285                   Mt.
1296                   Mt.
1325          Justin&Vicky
1422                JoAnn,
1630          Justin&Vicky
1735                A-Team
1807                Ellen/
2183               Chia-Yu
2294           Twenty-Nine
2295           Twenty-Nine
2459                A-Team
2460                A-Team
2500                A-Team
3502                Céline
3517           Marie-Josee
3701                 2Plus
4075                  K.J.
4138             Yeuan-Wen
4266            Jim&Brooke
4450                 2Plus
4667                J'Aime
4675                J'Aime
4800        Jean-Sébastien
4928    Isabella-And-Akahi
5553               Aurélia
5745                 Aimée
5755                I-Miun
5889             Yuan-Chou
6156                 2Plus
6263        Jean-Sébastien
Name: host_name, dtype: object


In [58]:
# investigate thhis name further
# Get the rows with non-letter values
rows_with_non_letters = data_without_nan_nan_rpm_and_format_license[non_letter_mask]

# Print the rows with non-letter values
print(f"Rows with non-letter values: {rows_with_non_letters['host_name']}")

Rows with non-letter values: 191                 Jean-Sébastien
1196                Jean-Sébastien
1274                Mt. Baker Home
1277                Mt. Baker Home
1285                Mt. Baker Home
1296                Mt. Baker Home
1325                  Justin&Vicky
1422    JoAnn, Marilyn, And Bernie
1630                  Justin&Vicky
1735                        A-Team
1807                    Ellen/ Tom
2183                       Chia-Yu
2294            Twenty-Nine Nights
2295            Twenty-Nine Nights
2459                        A-Team
2460                        A-Team
2500                        A-Team
3502                 Céline & Kurt
3517                   Marie-Josee
3701                         2Plus
4075                          K.J.
4138                     Yeuan-Wen
4266                    Jim&Brooke
4450                         2Plus
4667                        J'Aime
4675                        J'Aime
4800                Jean-Sébastien
4928            Isabella-A

In [60]:
# function to convert name into assumed gender
def get_assumed_gender(name):
    d = gender.Detector()
    assumed_gender = d.get_gender(name)

    if assumed_gender == 'male':
        return 'male'
    elif assumed_gender == 'female':
        return 'female'
    else:
        return 'unknown'

**Note:** used for ideas https://pypi.org/project/gender-guesser/

In [ ]:
#name_data_processed['host_gender'] = name_data_processed['host_name'].apply(get_assumed_gender)

In [62]:
name_data_processed.loc[~non_letter_mask, 'assumed_gender'] = name_data_processed.loc[~non_letter_mask, 'host_name'].apply(get_assumed_gender)

name_data_processed.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,name_processed,assumed_gender
0,879181,In Jimi Hendrix's neighborhood 3,287172,Laura,Central Area,Atlantic,47.60096,-122.30190,Private room,53,30,99,2022-12-16,0.80,2,247,4,stropli19002301,jimi hendrix neighborhood 3,female
1,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,159,2022-09-03,0.95,3,99,2,stropli19002622,fab privat seattl urban cottag,female
2,902609,Entire guest suite in West Seattle,3264544,Fabio,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,2,238,2023-03-21,1.95,5,206,27,stropli19002360,entir guest suit inest seattl,male
3,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,2,175,2023-01-15,1.14,9,356,20,stropli19003039,gloriou sun room memori foamb,unknown
4,226536,Serene Room in Sunny Cottage near Discovery Park,209571,Cheryl,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,1,316,2022-12-12,2.28,1,209,53,stropli19000684,seren room sunni cottag near discoveri park,female


In [64]:
name_data_processed['assumed_gender'].value_counts()

female     2049
male       1424
unknown    1410
Name: assumed_gender, dtype: int64

In [67]:
unknown_gender_rows = name_data_processed.loc[name_data_processed['assumed_gender'] == 'unknown']
unknown_gender_rows['host_name'].head(20)

3         Angielena
5         Angielena
7             Errin
32            Saint
40            Trung
44           Leslie
54     SeattleStays
62          Shannon
78        Angielena
94              Nan
100           Kelly
102           Shawn
106    SeattleStays
109             Lee
110       Angielena
112           Jamie
115           Sandy
121           Kelly
124          Gerrit
138         Vaibhav
Name: host_name, dtype: object

In [74]:
# I will make an assumption based on my observations and domain knowledge
# if name ends on 'a' and 'y' then this is female
# if name ends on anythings else it is male
name_data_processed.loc[name_data_processed['assumed_gender'] \
                        == 'unknown', 'assumed_gender'] \
= name_data_processed.loc[name_data_processed['assumed_gender'] \
                          == 'unknown', 'name_processed'].apply(lambda name: 'female' \
                                                                if name[-1] in ['a', 'y', 'i'] else 'male')


name_data_processed['assumed_gender'].value_counts()

male      2783
female    2100
Name: assumed_gender, dtype: int64

In [75]:
name_data_processed.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                       373
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
name_processed                      0
assumed_gender                     34
dtype: int64

In [79]:
name_data_processed['assumed_gender'].isnull().sum()

34

In [84]:
# Manually update gender values for rows with non-letter names

# I made assumptions such as:
# 191 Jean-Sébastien is male
# 1196 Jean-Sébastien  is male
# 1274 Mt. is group
# 1277 Mt. is group
# 1285 Mt. is group
# 1296 Mt. is group
# 1325 Justin&Vicky is group
# 1422 JoAnn, is group
# 1630 Justin&Vicky is group
# 1735 A-Team is group
# 1807 Ellen/ is group
# 2183 Chia-Yu  is female
# 2294 Twenty-Nine is group
# 2295 Twenty-Nine is group
# 2459 A-Team is group
# 2460 A-Team is group
# 2500 A-Team is group
# 3502 Céline is female
# 3517 Marie-Josee is female
# 3701 2Plus is group
# 4075 K.J.  is male
# 4138 Yeuan-Wen is male
# 4266 Jim&Brooke is group
# 4450 2Plus is group
# 4667 J'Aime is male
# 4675 J'Aime is male
# 4800 Jean-Sébastien  is male
# 4928 Isabella-And-Akahi is group
# 5553 Aurélia is female
# 5745 Aimée is female
# 5755 I-Miun is male
# 5889 Yuan-Chou is male
# 6156 2Plus is group
# 6263 Jean-Sébastien  is male
# 3701 2Plus is group
# 4075 K.J.  is male
# 4138 Yeuan-Wen is male
# 4266 Jim&Brooke is group
# 4450 2Plus is group
# 4667 J'Aime is male
# 4675 J'Aime is male
# 4800 Jean-Sébastien is male
# 4928 Isabella-And-Akahi is group
# 5553 Aurélia is female
# 5745 Aimée is female
# 5755 I-Miun is male
# 5889 Yuan-Chou is male
# 6156 2Plus is group
# 6263 Jean-Sébastien is male

# However, we do not ant to creat additional variables such as group 
# so, just use the same logic as before with 'a', 'y'
name_data_processed.loc[name_data_processed['assumed_gender'].isnull(), 'assumed_gender'] = [
    'female' if name.endswith(('a', 'y')) \
    else 'male' for name in name_data_processed.loc[name_data_processed['assumed_gender'].isnull(), 'host_name']
]

name_data_processed.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                       373
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
name_processed                      0
assumed_gender                      0
dtype: int64

**Note:** This code works. At first I was planning to manually add assumed_gender for 34 unique names but then i reallized that there is 1410 'unknown' so I ended up making an assumption that is word end with 'a', 'y' the female and everyone else is male.

## Handling Inconsistent or Invalid Values

In [85]:
# Create an empty dictionary to store the host listings count
host_listings_count = {}

# use for loop to iterate over each row
for index, row in data_without_nan_nan_rpm_and_format_license.iterrows():
    host_id = row['host_id']
    calculated_count = row['calculated_host_listings_count']
    
    # if the host_id is not already in the dictionary, add it and set the count to 0
    if host_id not in host_listings_count:
        host_listings_count[host_id] = 0
    
    # else add 1
    host_listings_count[host_id] += 1

correct_count = 0
incorrect_count = 0

# Compare the calculated count with the actual count for each host
for host_id, calculated_count in data_without_nan_nan_rpm_and_format_license[['host_id', 'calculated_host_listings_count']].values:
    if host_listings_count[host_id] == calculated_count:
        correct_count += 1
    else:
        incorrect_count += 1

print(f'Number of hosts with correct calculated_host_listings_count: {correct_count}')
print(f'Number of hosts with incorrect calculated_host_listings_count: {incorrect_count}')

Number of hosts with correct calculated_host_listings_count: 4406
Number of hosts with incorrect calculated_host_listings_count: 511


**Note:** There is a chance that the difference is due to human error or hosts having other properties outside of Seattle. For this project, we will assume that the rental market in Seattle is unique. Therefore, I will create a new column 'total_count_of_hosts_listings' and recalculate the total number of host listings, storing it in the new column. Subsequently, I will drop the 'calculated_host_listings_count' column.

In [86]:
# Calculate the total number of host listings for each host_id
total_listings_count = name_data_processed.groupby('host_id')['id'].count()

In [87]:
# Create a new column 'total_of_host_listings_count' with the recalculated total listings count
name_data_processed['total_of_host_listings_count'] = name_data_processed['host_id'].map(total_listings_count)

name_data_processed.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,name_processed,assumed_gender,total_of_host_listings_count
0,879181,In Jimi Hendrix's neighborhood 3,287172,Laura,Central Area,Atlantic,47.60096,-122.30190,Private room,53,...,99,2022-12-16,0.80,2,247,4,stropli19002301,jimi hendrix neighborhood 3,female,2
1,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,...,159,2022-09-03,0.95,3,99,2,stropli19002622,fab privat seattl urban cottag,female,3
2,902609,Entire guest suite in West Seattle,3264544,Fabio,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,...,238,2023-03-21,1.95,5,206,27,stropli19002360,entir guest suit inest seattl,male,5
3,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,...,175,2023-01-15,1.14,9,356,20,stropli19003039,gloriou sun room memori foamb,male,9
4,226536,Serene Room in Sunny Cottage near Discovery Park,209571,Cheryl,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,...,316,2022-12-12,2.28,1,209,53,stropli19000684,seren room sunni cottag near discoveri park,female,1


## Removing Unnecessary Columns (and finishing dealing with missing values)

In [88]:
# making a copy of the data set and then dropping off columns 
# so, that if we need to go back we have a copu with all columns
processed_data = name_data_processed.copy()

processed_data.shape

(4917, 21)

In [89]:
processed_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,name_processed,assumed_gender,total_of_host_listings_count
0,879181,In Jimi Hendrix's neighborhood 3,287172,Laura,Central Area,Atlantic,47.60096,-122.30190,Private room,53,...,99,2022-12-16,0.80,2,247,4,stropli19002301,jimi hendrix neighborhood 3,female,2
1,6606,"Fab, private seattle urban cottage!",14942,Joyce,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,...,159,2022-09-03,0.95,3,99,2,stropli19002622,fab privat seattl urban cottag,female,3
2,902609,Entire guest suite in West Seattle,3264544,Fabio,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,...,238,2023-03-21,1.95,5,206,27,stropli19002360,entir guest suit inest seattl,male,5
3,9419,Glorious sun room w/ memory foambed,30559,Angielena,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,...,175,2023-01-15,1.14,9,356,20,stropli19003039,gloriou sun room memori foamb,male,9
4,226536,Serene Room in Sunny Cottage near Discovery Park,209571,Cheryl,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,...,316,2022-12-12,2.28,1,209,53,stropli19000684,seren room sunni cottag near discoveri park,female,1


In [90]:
processed_data.isnull().sum()

id                                  0
name                                0
host_id                             0
host_name                           0
neighbourhood_group                 0
neighbourhood                       0
latitude                            0
longitude                           0
room_type                           0
price                               0
minimum_nights                      0
number_of_reviews                   0
last_review                       373
reviews_per_month                   0
calculated_host_listings_count      0
availability_365                    0
number_of_reviews_ltm               0
license                             0
name_processed                      0
assumed_gender                      0
total_of_host_listings_count        0
dtype: int64

### Dropping 'name' column

In [91]:
processed_data.drop('name', axis = 1, inplace = True)

In [92]:
processed_data.shape

(4917, 20)

### Dropping 'license' column

In [93]:
processed_data.drop('license', axis = 1, inplace = True)

In [94]:
processed_data.shape

(4917, 19)

### Dropping 'host_name' column

In [95]:
processed_data.drop('host_name', axis = 1, inplace = True)

In [96]:
processed_data.shape

(4917, 18)

### Dropping 'last_review' column

In [97]:
# after careful consideration of current problem statement,
# I believe that we do not need the 'last_review' column.
# since our analysis is not based on time, but rather on the other features,
# let's drop the 'last_review' column.
processed_data.drop('last_review', axis = 1, inplace = True)

In [98]:
processed_data.shape

(4917, 17)

### Dropping 'calculated_host_listings_count' column

In [99]:
processed_data.drop('calculated_host_listings_count', axis = 1, inplace = True)

In [100]:
processed_data.shape

(4917, 16)

## Overview Cleaned Data

In [101]:
processed_data.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,number_of_reviews_ltm,name_processed,assumed_gender,total_of_host_listings_count
0,879181,287172,Central Area,Atlantic,47.60096,-122.30190,Private room,53,30,99,0.80,247,4,jimi hendrix neighborhood 3,female,2
1,6606,14942,Other neighborhoods,Wallingford,47.65444,-122.33629,Entire home/apt,99,30,159,0.95,99,2,fab privat seattl urban cottag,female,3
2,902609,3264544,Delridge,Highland Park,47.52383,-122.35287,Entire home/apt,70,2,238,1.95,206,27,entir guest suit inest seattl,male,5
3,9419,30559,Other neighborhoods,Georgetown,47.55017,-122.31937,Private room,55,2,175,1.14,356,20,gloriou sun room memori foamb,male,9
4,226536,209571,Magnolia,Lawton Park,47.66133,-122.40088,Private room,61,1,316,2.28,209,53,seren room sunni cottag near discoveri park,female,1


In [102]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4917 entries, 0 to 6375
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            4917 non-null   int64  
 1   host_id                       4917 non-null   int64  
 2   neighbourhood_group           4917 non-null   object 
 3   neighbourhood                 4917 non-null   object 
 4   latitude                      4917 non-null   float64
 5   longitude                     4917 non-null   float64
 6   room_type                     4917 non-null   object 
 7   price                         4917 non-null   int64  
 8   minimum_nights                4917 non-null   int64  
 9   number_of_reviews             4917 non-null   int64  
 10  reviews_per_month             4917 non-null   float64
 11  availability_365              4917 non-null   int64  
 12  number_of_reviews_ltm         4917 non-null   int64  
 13  nam

In [103]:
processed_data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,number_of_reviews_ltm,total_of_host_listings_count
count,4.917000e+03,4.917000e+03,4917.000000,4917.000000,4917.000000,4917.000000,4917.000000,4917.000000,4917.000000,4917.000000,4917.000000
mean,2.403211e+17,1.358716e+08,47.626225,-122.334410,166.208664,6.233272,77.559487,2.230026,174.560301,22.170632,8.621721
std,3.360253e+17,1.548640e+08,0.050967,0.034400,165.978737,11.669780,109.382234,1.936660,118.529493,24.378791,16.667258
min,6.606000e+03,5.887000e+03,47.497920,-122.417540,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.441043e+07,1.401522e+07,47.595950,-122.358650,94.000000,1.000000,9.000000,0.710000,71.000000,3.000000,1.000000
50%,4.967975e+07,6.504893e+07,47.625670,-122.332510,131.000000,2.000000,35.000000,1.800000,161.000000,15.000000,2.000000
75%,6.394425e+17,2.171068e+08,47.665530,-122.309559,193.000000,3.000000,99.000000,3.280000,281.000000,34.000000,6.000000
max,8.527981e+17,5.034165e+08,47.734010,-122.241490,4000.000000,180.000000,1251.000000,22.700000,365.000000,406.000000,91.000000


In [104]:
processed_data.isnull().sum()

id                              0
host_id                         0
neighbourhood_group             0
neighbourhood                   0
latitude                        0
longitude                       0
room_type                       0
price                           0
minimum_nights                  0
number_of_reviews               0
reviews_per_month               0
availability_365                0
number_of_reviews_ltm           0
name_processed                  0
assumed_gender                  0
total_of_host_listings_count    0
dtype: int64

## Saving Cleaned Data as a new CSV file

In [105]:
# save cleaned dataframe
processed_data.to_csv('../data/cleaned_data.csv', index = False)